# IMDB dataset. Experiment 3

This third experiment consist in evaluate the performance of a neural network with one layer on the IMDB classification reviews problem. We have selected `MlFlow` to track the performance variation an assess the models.

In this case, the activation function is `relu`.

Then, we import the necessary libraries.

In [1]:
# imports
import os

import tensorflow as tf
import numpy as np
import mlflow

from mlflow.tracking import MlflowClient

2024-04-29 12:46:51.908507: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 12:46:51.983279: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 12:46:52.217446: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 12:46:53.572139: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Here we add the path to the local module so we can import our own funtion to the notebook.

In [2]:
# import local modules
import sys
sys.path.append('/mnt/0A2AAC152AABFBB7/sideProjects/deepLearning')
from deeplearning.deep_utils import imdb_model_setup_one_layer

From this point we do not add more comments since we follow the path determined in Chollet's book.

In [3]:
# Loading the IMDB dataset

(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=10000)

In [4]:
# Encoding the integer sequences via multi-hot encoding

def vectorize_sequences(sequences, dimension=10000):

    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

In [5]:
# Setting aside a validation set

x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [6]:
# Initialize client and experiment

EXPERIMENT_NAME = "deepLearning_IMDB_dataset"
client = MlflowClient()
mlflow.set_experiment(EXPERIMENT_NAME)
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

We create a list with the number of units in the layer. It would be interesting to try with more units but due to Collab limitation we can only experiment with three possible sizes.

In [7]:
units_2_hidden = [8, 16, 32]

Here we train the three NN and store their performances with the help of `MlFlow`.

In [8]:
count = 0
for units in units_2_hidden:

    model, history = imdb_model_setup_one_layer(
        units_layer=units,
        activ_func="relu",
        X_train=partial_x_train,
        y_train=partial_y_train,
        X_val=x_val,
        y_val=y_val
    )

    hist_dict = history.history
    test_loss, test_accuracy = model.evaluate(x_test, y_test)


    with mlflow.start_run():
        mlflow.set_tag("model", "Base_tanh_{}".format(count))
        mlflow.log_param("units_1st_layer", units)
        mlflow.log_param("hidden_activation", 'relu')
        mlflow.log_param("epochs", 4)
        mlflow.log_param("batch_size", 512)
        mlflow.log_metric("accuracy", test_accuracy)
        mlflow.log_metric("loss", test_loss)
    count+=1

    del hist_dict
    del history
    del model

Epoch 1/4


2024-04-29 12:47:01.940643: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 600000000 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6923 - loss: 0.6115 - val_accuracy: 0.8591 - val_loss: 0.4284
Epoch 2/4


2024-04-29 12:47:03.485950: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 400000000 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8893 - loss: 0.3784 - val_accuracy: 0.8751 - val_loss: 0.3558
Epoch 3/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9127 - loss: 0.2923 - val_accuracy: 0.8808 - val_loss: 0.3179
Epoch 4/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9230 - loss: 0.2478 - val_accuracy: 0.8888 - val_loss: 0.2919
115/782 ━━━━━━━━━━━━━━━━━━━━ 0s 890us/step - accuracy: 0.8941 - loss: 0.2947

...

782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 825us/step - accuracy: 0.8822 - loss: 0.3059
Epoch 1/4


2024-04-29 12:47:09.468679: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 600000000 exceeds 10% of free system memory.


28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7040 - loss: 0.5902

2024-04-29 12:47:10.951105: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 400000000 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.7121 - loss: 0.5823 - val_accuracy: 0.8680 - val_loss: 0.3939
Epoch 2/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8939 - loss: 0.3432 - val_accuracy: 0.8735 - val_loss: 0.3374
Epoch 3/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9152 - loss: 0.2691 - val_accuracy: 0.8907 - val_loss: 0.2949
Epoch 4/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9288 - loss: 0.2254 - val_accuracy: 0.8891 - val_loss: 0.2846
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8796 - loss: 0.2975
Epoch 1/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.7130 - loss: 0.5720 - val_accuracy: 0.8651 - val_loss: 0.3815
Epoch 2/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8923 - loss: 0.3290 - val_accuracy: 0.8843 - val_loss: 0.3133
Epoch 3/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9186 - loss: 0.2530 - val_accuracy: 0.8892 - val_loss: 0.2868
Epoch 4/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - a

In [10]:
runs = mlflow.search_runs()
PATH_TO_RUNS = "/mnt/0A2AAC152AABFBB7/sideProjects/deepLearning/mlflow_runs"
runs.to_csv(
    os.path.join(PATH_TO_RUNS, "imdb_runs_2.csv")
)
runs[['experiment_id', 'status', 'metrics.loss', 'metrics.accuracy', 'params.units_1st_layer', "params.hidden_activation"]]

,experiment_id,status,metrics.loss,metrics.accuracy,params.units_1st_layer,params.hidden_activation
0,926124755463801135,FINISHED,0.288918,0.88404,32,relu
1,926124755463801135,FINISHED,0.297910,0.88084,16,relu
2,926124755463801135,FINISHED,0.304674,0.88280,8,relu
3,926124755463801135,FINISHED,0.338135,0.87084,64,tanh
4,926124755463801135,FINISHED,0.309813,0.87840,32,tanh
5,926124755463801135,FINISHED,0.287834,0.88152,16,tanh
6,926124755463801135,FINISHED,0.385586,0.84884,64,relu
7,926124755463801135,FINISHED,0.313741,0.87392,32,relu
8,926124755463801135,FINISHED,0.294502,0.88060,16,relu


In [11]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,metrics.loss,params.batch_size,params.epochs,params.units_1st_layer,params.hidden_activation,params.units_2nd_layer,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.user,tags.model,tags.mlflow.source.type
0,b3288fe1ad6e4c269fdda3ec7897bdae,926124755463801135,FINISHED,file:///mnt/0A2AAC152AABFBB7/sideProjects/deep...,2024-04-29 10:47:21.145000+00:00,2024-04-29 10:47:21.169000+00:00,0.88404,0.288918,512,4,32,relu,None,/mnt/0A2AAC152AABFBB7/sideProjects/deepLearnin...,inquisitive-loon-101,luisggon,Base_tanh_2,LOCAL
1,913cf68f86e443adae0b87269c2d7897,926124755463801135,FINISHED,file:///mnt/0A2AAC152AABFBB7/sideProjects/deep...,2024-04-29 10:47:15.278000+00:00,2024-04-29 10:47:15.303000+00:00,0.88084,0.297910,512,4,16,relu,None,/mnt/0A2AAC152AABFBB7/sideProjects/deepLearnin...,spiffy-shrimp-14,luisggon,Base_tanh_1,LOCAL
2,d1c1615ba1a44a92a80dd6e88b0b1af4,926124755463801135,FINISHED,file:///mnt/0A2AAC152AABFBB7/sideProjects/deep...,2024-04-29 10:47:09.139000+00:00,2024-04-29 10:47:09.164000+00:00,0.88280,0.304674,512,4,8,relu,None,/mnt/0A2AAC152AABFBB7/sideProjects/deepLearnin...,unruly-gull-578,luisggon,Base_tanh_0,LOCAL
3,2143f77529164b8ca07fca057f2f5a08,926124755463801135,FINISHED,file:///mnt/0A2AAC152AABFBB7/sideProjects/deep...,2024-04-29 10:29:48.952000+00:00,2024-04-29 10:29:49.003000+00:00,0.87084,0.338135,512,4,64,tanh,64,/mnt/0A2AAC152AABFBB7/sideProjects/deepLearnin...,likeable-deer-618,luisggon,Base_tanh_2,LOCAL
4,fd6c45e187074f2680867d44a7fd7738,926124755463801135,FINISHED,file:///mnt/0A2AAC152AABFBB7/sideProjects/deep...,2024-04-29 10:29:40.953000+00:00,2024-04-29 10:29:40.979000+00:00,0.87840,0.309813,512,4,32,tanh,32,/mnt/0A2AAC152AABFBB7/sideProjects/deepLearnin...,sassy-colt-222,luisggon,Base_tanh_1,LOCAL
5,86caf32d83274909b20505b6d3e74afe,926124755463801135,FINISHED,file:///mnt/0A2AAC152AABFBB7/sideProjects/deep...,2024-04-29 10:29:34.243000+00:00,2024-04-29 10:29:34.275000+00:00,0.88152,0.287834,512,4,16,tanh,16,/mnt/0A2AAC152AABFBB7/sideProjects/deepLearnin...,shivering-mouse-746,luisggon,Base_tanh_0,LOCAL
6,e1eee968bf3d4c169386b71c68da99c2,926124755463801135,FINISHED,file:///mnt/0A2AAC152AABFBB7/sideProjects/deep...,2024-04-29 09:47:10.132000+00:00,2024-04-29 09:47:10.159000+00:00,0.84884,0.385586,512,4,64,relu,64,/mnt/0A2AAC152AABFBB7/sideProjects/deepLearnin...,upset-snake-27,luisggon,Base_relu_2,LOCAL
7,acccca3ed4d24086bf23482cb15cc9d8,926124755463801135,FINISHED,file:///mnt/0A2AAC152AABFBB7/sideProjects/deep...,2024-04-29 09:47:02.683000+00:00,2024-04-29 09:47:02.711000+00:00,0.87392,0.313741,512,4,32,relu,32,/mnt/0A2AAC152AABFBB7/sideProjects/deepLearnin...,receptive-midge-765,luisggon,Base_relu_1,LOCAL
8,be8220fcea6547c4b537882eed3e31ee,926124755463801135,FINISHED,file:///mnt/0A2AAC152AABFBB7/sideProjects/deep...,2024-04-29 09:46:56.301000+00:00,2024-04-29 09:46:56.329000+00:00,0.88060,0.294502,512,4,16,relu,16,/mnt/0A2AAC152AABFBB7/sideProjects/deepLearnin...,colorful-skink-172,luisggon,Base_relu_0,LOCAL
